In [4]:
%cd D:\z_old_projects\talking_data\scripts

D:\z_old_projects\talking_data\scripts


In [5]:
ls

 Volume in drive D is WD500
 Volume Serial Number is 3E8A-ADD9

 Directory of D:\z_old_projects\talking_data\scripts

05/09/2018  05:45 PM    <DIR>          .
05/09/2018  05:45 PM    <DIR>          ..
05/08/2018  02:15 AM             1,657 blend.ipynb
05/10/2018  10:14 PM            72,310 FE.jpg
05/12/2018  01:56 AM            19,002 lgbm.py
04/24/2018  07:43 PM            51,853 xgb_9756.ipynb
               4 File(s)        144,822 bytes
               2 Dir(s)  181,189,406,720 bytes free


In [6]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
import gc 

In [157]:
NCHUNK = 35000000
OFFSET = 78000000
val_size= 5000000
nrows=184903890
nchunk=NCHUNK
frm=nrows-OFFSET
to=frm+nchunk

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        }

train_df = pd.read_csv("../input/train.csv",
                       skiprows=range(1,frm), nrows=to-frm, dtype=dtypes, 
                       usecols=['ip','app','device','os'])

test_df = pd.read_csv("../input/test.csv", dtype=dtypes, usecols=['ip','app','device','os'])

In [158]:
train_df = train_df.append(test_df)
train_df.shape

(53790469, 4)

In [159]:
# hashing users 
train_df['user'] = np.dot(train_df[['os', 'device', 'ip']].values, [10**8, 10**6, 1])

In [160]:
tsvd = TruncatedSVD(3)
cross1 = np.log1p(pd.crosstab(train_df['ip'], train_df['app']))
cross2 = np.log1p(pd.crosstab(train_df['user'], train_df['app']))
factors1 = tsvd.fit_transform(cross1)
factors2 = tsvd.fit_transform(cross2)

In [161]:
factors1 = pd.DataFrame(np.hstack([cross1.index.values.reshape(-1,1), factors1]), 
                        columns=['ip', 'fac1_1', 'fac1_2', 'fac1_3'])
factors2 = pd.DataFrame(np.hstack([cross2.index.values.reshape(-1,1), factors2]), 
                        columns=['user', 'fac2_1', 'fac2_2', 'fac2_3'])

In [162]:
train_df = train_df.merge(factors1, on=['ip'], how='left')
train_df = train_df.merge(factors2, on=['user'], how='left')

In [163]:
train_df[['fac1_1', 'fac1_2', 'fac1_3']].to_feather('../dumps/factor1_SMALL.feather')
train_df[['fac2_1', 'fac2_2', 'fac2_3']].to_feather('../dumps/factor2_SMALL.feather')

In [164]:
pd.read_feather('../dumps/factor1_SMALL.feather').shape

(53790469, 3)

In [7]:
dump_name = '_SMALL'
print('Load dumps...')
train_df = pd.read_feather('../dumps/kaggle_train' + dump_name + '.feather').set_index('index')
print(train_df.shape)
val_df = pd.read_feather('../dumps/kaggle_val' + dump_name + '.feather').set_index('index')
print(val_df.shape)

Load dumps...
(30000000, 27)
(5000000, 27)


In [9]:
factor1 = pd.read_feather('../dumps/factor1' + dump_name + '.feather')
factor2 = pd.read_feather('../dumps/factor2' + dump_name + '.feather')

In [10]:
factors = pd.concat([factor1, factor2], axis=1)
del factor1, factor2
gc.collect()

308

In [12]:
ntrain = train_df.shape[0]
nfull = train_df.shape[0] + val_df.shape[0]

In [13]:
train_df = pd.concat([train_df, factors[:ntrain]], axis=1)

In [14]:
val_df = pd.concat([val_df, factors[ntrain:nfull]], axis=1)

In [15]:
factors_test = factors[nfull:]

In [16]:
del factors
gc.collect()

2500